In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



/kaggle/input/stack-overflow-annual-developer-survey-2024/survey_results_schema.csv
/kaggle/input/stack-overflow-annual-developer-survey-2024/2024 Developer Survey.pdf
/kaggle/input/stack-overflow-annual-developer-survey-2024/survey_results_public.csv


In [12]:
results_schema = pd.read_csv('/kaggle/input/stack-overflow-annual-developer-survey-2024/survey_results_public.csv')
results_schema.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,ResponseId,MainBranch,Age,Employment,RemoteWork,Check,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,SurveyLength,SurveyEase,ConvertedCompYearly,JobSat
0,1,I am a developer by profession,Under 18 years old,"Employed, full-time",Remote,Apples,Hobby,Primary/elementary school,Books / Physical media,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,3,I am a developer by profession,45-54 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,...,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,NaN,NaN
3,4,I am learning to code,18-24 years old,"Student, full-time",NaN,Apples,NaN,Some college/university study without earning ...,"Other online resources (e.g., videos, blogs, f...",Stack Overflow;How-to videos;Interactive tutorial,...,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Easy,NaN,NaN
4,5,I am a developer by profession,18-24 years old,"Student, full-time",NaN,Apples,NaN,"Secondary school (e.g. American high school, G...","Other online resources (e.g., videos, blogs, f...",Technical documentation;Blogs;Written Tutorial...,...,NaN,NaN,NaN,NaN,NaN,NaN,Too short,Easy,NaN,NaN


In [13]:
results_schema.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65437 entries, 0 to 65436
Columns: 114 entries, ResponseId to JobSat
dtypes: float64(13), int64(1), object(100)
memory usage: 56.9+ MB


In [14]:
print(results_schema.columns)

Index(['ResponseId', 'MainBranch', 'Age', 'Employment', 'RemoteWork', 'Check',
       'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline',
       ...
       'JobSatPoints_6', 'JobSatPoints_7', 'JobSatPoints_8', 'JobSatPoints_9',
       'JobSatPoints_10', 'JobSatPoints_11', 'SurveyLength', 'SurveyEase',
       'ConvertedCompYearly', 'JobSat'],
      dtype='object', length=114)


The data takes up 56.9+ MBs of data. Looking through the 2024 Developers Survey, there are many columns that probably dont correlate to the expected Salary Forecast. For now, I will start reducing the data size by getting rid of the bad responses. To do this, I will use one of the control questions checking for respondant attention to weed out inaccurate data points.

In [15]:
results_schema = results_schema[results_schema["Check"] == "Apples"]
results_schema.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65437 entries, 0 to 65436
Columns: 114 entries, ResponseId to JobSat
dtypes: float64(13), int64(1), object(100)
memory usage: 56.9+ MB


It looks like the survey already took out all the checked data. We can get rid of the column. Next, I want to take a look at how the "select all that apply" questions were stored in the dataframe

In [16]:
results_schema = results_schema.drop('Check', axis=1)
print(list(results_schema.columns))

['ResponseId', 'MainBranch', 'Age', 'Employment', 'RemoteWork', 'CodingActivities', 'EdLevel', 'LearnCode', 'LearnCodeOnline', 'TechDoc', 'YearsCode', 'YearsCodePro', 'DevType', 'OrgSize', 'PurchaseInfluence', 'BuyNewTool', 'BuildvsBuy', 'TechEndorse', 'Country', 'Currency', 'CompTotal', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith', 'LanguageAdmired', 'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith', 'DatabaseAdmired', 'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'PlatformAdmired', 'WebframeHaveWorkedWith', 'WebframeWantToWorkWith', 'WebframeAdmired', 'EmbeddedHaveWorkedWith', 'EmbeddedWantToWorkWith', 'EmbeddedAdmired', 'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith', 'MiscTechAdmired', 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith', 'ToolsTechAdmired', 'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', 'NEWCollabToolsAdmired', 'OpSysPersonal use', 'OpSysProfessional use', 'OfficeStackAsyncHaveWorkedWith', 'OfficeStackAsyncWantToWorkWith', 'OfficeS

Looking through the survey, there are quite a few data points that dont relate to compensation. 
* SurveyLength
* SurveyEase
* PurchaseInfluence
* BuyNewTool
* TechEndorse
* LanguageWantToWorkWith
* LanguageAdmired
* DatabaseWantToWorkWith
* DatabaseAdmired
* PlatformWantToWorkWith
* PlatformAdmired
* WebframeWantToWorkWith
* WebframeAdmired
* EmbeddedWantToWorkWith
* EmbeddedAdmired
* MiscTechWantToWorkWith
* MiscTechAdmired
* ToolsTechWantToWorkWith
* ToolsTechAdmired
* NEWCollabToolsWantToWorkWith
* NEWColabToolsAdmired
* OpSysPersonal use
* OfficeStackAsyncWantToWorkWith
* OfficeStackSyncAdmired


We will also need to restrict the datapool to those who responded with "I am a developer by profession" and "I am not primarily a developer, but I write code sometimes as part of my work/studies".



In [17]:
columns_to_drop = ['SurveyLength','SurveyEase','PurchaseInfluence',
                   'BuyNewTool','TechEndorse','LanguageWantToWorkWith',
                  'LanguageAdmired','DatabaseWantToWorkWith','DatabaseAdmired',
                  'PlatformAdmired','WebframeWantToWorkWith','WebframeAdmired',
                  'EmbeddedWantToWorkWith','EmbeddedAdmired','MiscTechWantToWorkWith',
                  'MiscTechAdmired','ToolsTechWantToWorkWith','ToolsTechAdmired',
                  'NEWCollabToolsWantToWorkWith','NEWCollabToolsAdmired','OpSysPersonal use',
                  'OfficeStackAsyncWantToWorkWith','OfficeStackSyncAdmired']
model_data = results_schema.drop(columns_to_drop, axis=1)
model_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65437 entries, 0 to 65436
Data columns (total 90 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ResponseId                      65437 non-null  int64  
 1   MainBranch                      65437 non-null  object 
 2   Age                             65437 non-null  object 
 3   Employment                      65437 non-null  object 
 4   RemoteWork                      54806 non-null  object 
 5   CodingActivities                54466 non-null  object 
 6   EdLevel                         60784 non-null  object 
 7   LearnCode                       60488 non-null  object 
 8   LearnCodeOnline                 49237 non-null  object 
 9   TechDoc                         40897 non-null  object 
 10  YearsCode                       59869 non-null  object 
 11  YearsCodePro                    51610 non-null  object 
 12  DevType                         

Now that we have it down to 90 columns, the data types are fully visible. There are varying counts on the non-null section. I will need to change those later on for my model. For now there are some more columns that dont correlate to total compensation to keep removing.

* OfficeStackAsyncAdmired
* OfficeStackSyncWantToWorkWith
* AISearchDevWantToWorkWith
* AISearchDevAdmired
* AIBen
* AIAcc
* AIComplex
* AIToolInterest in Using
* AIToolNot interested in Using
* AINextMuch more integrated
* AINextNo change
* AINextMore integrated
* AINextLess integrated
* AINextMuch less integrated
* AIThreat
* AIEthics
* AIChallenges
* ICorPM
* Knowledge_1
* Knowledge_2
* Knowledge_3
* Knowledge_4
* Knowledge_5
* Knowledge_6
* Knowledge_7
* Knowledge_8
* Knowledge_9
* Frequency_1
* Frequency_2
* Frequency_3
* TimeSearching
* TimeAnswering
* Frustration
* ProfessionalTech
* ProfessionalCloud
* ProfessionalQuestion
* JobSatPoints_1
* JobSatPoints_4
* JobSatPoints_5
* JobSatPoints_6
* JobSatPoints_7
* JobSatPoints_8
* JobSatPoints_9
* JobSat

In [18]:
columns_to_drop = ['OfficeStackAsyncAdmired','OfficeStackSyncWantToWorkWith','AISearchDevWantToWorkWith',
                  'AISearchDevAdmired','AIBen','AIAcc','AIComplex',
                  'AIToolNot interested in Using','AINextMuch more integrated','AINextNo change',
                  'AINextMore integrated','AINextLess integrated','AINextMuch less integrated',
                  'AIThreat','AIEthics','AIChallenges','ICorPM','Knowledge_1','Knowledge_2','Knowledge_3',
                  'Knowledge_4','Knowledge_5','Knowledge_6','Knowledge_7','Knowledge_8','Knowledge_9',
                  'Frequency_1','Frequency_2','Frequency_3','TimeSearching','TimeAnswering','Frustration',
                  'ProfessionalTech','ProfessionalCloud','ProfessionalQuestion','JobSatPoints_1','JobSatPoints_4',
                  'JobSatPoints_5','JobSatPoints_6','JobSatPoints_7','JobSatPoints_8','JobSatPoints_9',
                  'JobSat']
model_data = model_data.drop(columns_to_drop, axis=1)
model_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65437 entries, 0 to 65436
Data columns (total 47 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ResponseId                      65437 non-null  int64  
 1   MainBranch                      65437 non-null  object 
 2   Age                             65437 non-null  object 
 3   Employment                      65437 non-null  object 
 4   RemoteWork                      54806 non-null  object 
 5   CodingActivities                54466 non-null  object 
 6   EdLevel                         60784 non-null  object 
 7   LearnCode                       60488 non-null  object 
 8   LearnCodeOnline                 49237 non-null  object 
 9   TechDoc                         40897 non-null  object 
 10  YearsCode                       59869 non-null  object 
 11  YearsCodePro                    51610 non-null  object 
 12  DevType                         

In [19]:
columns_to_drop = ['AIToolInterested in Using','JobSatPoints_10','JobSatPoints_11',
                  'PlatformWantToWorkWith','BuildvsBuy']
model_data = model_data.drop(columns_to_drop, axis=1)
model_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65437 entries, 0 to 65436
Data columns (total 42 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ResponseId                      65437 non-null  int64  
 1   MainBranch                      65437 non-null  object 
 2   Age                             65437 non-null  object 
 3   Employment                      65437 non-null  object 
 4   RemoteWork                      54806 non-null  object 
 5   CodingActivities                54466 non-null  object 
 6   EdLevel                         60784 non-null  object 
 7   LearnCode                       60488 non-null  object 
 8   LearnCodeOnline                 49237 non-null  object 
 9   TechDoc                         40897 non-null  object 
 10  YearsCode                       59869 non-null  object 
 11  YearsCodePro                    51610 non-null  object 
 12  DevType                         

Now that the columns are down to what I want to analyze, I would like to take a look at each one to see if I can resolve missing data issues or reduce memory size by changing from object variables

       ResponseId                                         MainBranch  \
0               1                     I am a developer by profession   
1               2                     I am a developer by profession   
2               3                     I am a developer by profession   
3               4                              I am learning to code   
4               5                     I am a developer by profession   
...           ...                                                ...   
65427       65428  I am not primarily a developer, but I write co...   
65429       65430                              I am learning to code   
65430       65431                              I am learning to code   
65433       65434                     I am a developer by profession   
65436       65437                        I code primarily as a hobby   

                      Age                                         Employment  \
0      Under 18 years old                              

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
